In [1]:
import sys # aby importy z dołu działały
sys.path.append(r"C:\Users\piotr\OneDrive\____Moje-MOJE\MyProjects_4Fun\projects\World of Warcraft\python-etl")

In [2]:
from sqlalchemy import text, bindparam
from moduly.db_core import utworz_engine_do_db, _czy_duplikat
from moduly.services_persist_wynik import przefiltruj_dane_misji, zapisz_misje_dialogi_ai_do_db
from moduly.ai_gemini import zaladuj_api_i_klienta, instrukcja_tlumacz, instrukcja_redaktor
from moduly.utils import sklej_warunki_w_WHERE
from sqlalchemy.exc import IntegrityError
from scraper_wiki_main import parsuj_misje_z_url
from google import genai
import json
import zlib
import base64
from bs4 import BeautifulSoup
import pandas as pd
from pprint import pprint

In [ ]:
# silnik = utworz_engine_do_db()

In [3]:
parsuj_misje_z_url("https://warcraft.wiki.gg/wiki/Last_Bastion_of_the_Light")

{'Źródło': {'url': 'https://warcraft.wiki.gg/wiki/Last_Bastion_of_the_Light',
  'html_skompresowany': 'eJztPNuSG7ex7/yKNlM6lqvE2/CyXO7uqHTzJZGVjVaxfCqVmgJnmiS8GGACYMiln1z5gnyCv8Wf4i851ZgrydkVtyL7pKLVg5bT3Wg0Gj3dDaAx5xFfQyiYMRftNYZW6c5cRds28OiiTb9eKGlR2rbfOq+RSpVoLm1GZrjFq3Te9r/UKob3TIeaLSy859f8vBfxdd6USMOMm6O+HecdIn9I46RjVUeydYVkhTjxpuMIBJfXbVhpXFy0/xBvOitkUdv/YxonYBVItuZLZrmS5z32weYGmQ5X38gktRWLDJg1r6kj3jitdfIhZGqJN8Vzx+KNbft7LQqksBriTSdh2qDuqNRSjxBxfdEWVrdBMLm8aKNs++eWzQVCiEKYhIWc4MN2wZLLhZqrG4iYvs4I/5GisXN10wZjtwJpQnWEupM3ng2Tm7MNj+xq5nkY5z/B63fHGJ+dUX80Kv/c6qIPNldrdGpFTfgVhIpkkRdtr+yFRtthgi/lLERpUZ8tlLQdw3/E2cAbP8oeN8iXKzubKxGdJSyKSKBxckPdEsOCWcRNIth25kaUizjo9x+d5WMJlRAsMTgrfpztDbJ/J8eOVpu7CUjbZ2vUlodM5MOKeRQJLMTuaDcSEj6XrzaMXHFt/5zlltXb8Gve+5KFZIltsNxSp28wtZqJtn/O4yUwYSsQRMyyzoIL0jx1ddEeenWo6/WiPZi2IcJQkUwXbWa2MmxDvYVQLMMJ9uO2DUaHF+0ej9kSTS/vLBh63UQunzKc4KTfhop1zyezP+/RqMo/H0FtnlPWa2YsPGeGdAJqAXaF8Jrwex0Wf+yK/tNkm84MTagSvGhrtWkyQ4ELezZn4fVSq1RGZDRKz6xm0iRMo7Rnbf/KMm0zxjZy3qE+Xa9Zt